In [3]:
import pandas as pd
import numpy as np
import requests



In [4]:
# Get some data from the coinmarketcap api
url = "https://api.coinmarketcap.com/v2/ticker/"
params = dict(
    start=0,
    limit=100,
    sort='id'
)

coin_data = {}


r1 = requests.get(url, params=params)
r1_data = r1.json()

num_currencies = r1_data.get('metadata',{}).get('num_cryptocurrencies')
print(r1_data.get('data'))

for i in range(100,num_currencies,100):
    params['start'] = i
    r = requests.get(url, params=params)
    coin_data.update(r.json().get('data'))
    



{'1': {'id': 1, 'name': 'Bitcoin', 'symbol': 'BTC', 'website_slug': 'bitcoin', 'rank': 1, 'circulating_supply': 17359687.0, 'total_supply': 17359687.0, 'max_supply': 21000000.0, 'quotes': {'USD': {'price': 6341.33438475, 'volume_24h': 4448210565.04133, 'market_cap': 110083580082.0, 'percent_change_1h': 0.13, 'percent_change_24h': -0.24, 'percent_change_7d': -2.11}}, 'last_updated': 1541364861}, '2': {'id': 2, 'name': 'Litecoin', 'symbol': 'LTC', 'website_slug': 'litecoin', 'rank': 7, 'circulating_supply': 59021727.0, 'total_supply': 59021727.0, 'max_supply': 84000000.0, 'quotes': {'USD': {'price': 53.6060250692, 'volume_24h': 437143006.301715, 'market_cap': 3163920166.0, 'percent_change_1h': -0.33, 'percent_change_24h': 4.85, 'percent_change_7d': 3.49}}, 'last_updated': 1541364849}, '3': {'id': 3, 'name': 'Namecoin', 'symbol': 'NMC', 'website_slug': 'namecoin', 'rank': 414, 'circulating_supply': 14736400.0, 'total_supply': 14736400.0, 'max_supply': None, 'quotes': {'USD': {'price': 0.5

In [ ]:
coin_data

In [5]:
# Get a quick plot of market cap vs trading volume
cap_vol = []
for d in coin_data.values():
    cap_vol.append(dict(
        id=d['id'],
        name=d['name'],
        market_cap=d.get('quotes',{}).get('USD',{}).get('market_cap'),
        volume_24h=d.get('quotes',{}).get('USD',{}).get('volume_24h'),
        volatility_7d=d.get('quotes',{}).get('USD',{}).get('percent_change_7d')
    ))

cap_vol = pd.DataFrame.from_records(cap_vol)
cap_vol.head()

,id,market_cap,name,volatility_7d,volume_24h
0,269,4587492.0,AsiaCoin,0.60,9.552427
1,275,316327.0,PopularCoin,27.58,5.847085
2,276,184860.0,Bitstar,16.65,28.454324
3,278,57579.0,Quebecoin,4.08,4.062966
4,290,914937.0,BlueCoin,-23.50,1.592071


In [9]:
from bokeh.plotting import figure, show, output_notebook, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.transform import linear_cmap
from bokeh.palettes import Spectral6
output_notebook()

Loading BokehJS ...

In [12]:

p = figure(
    title="Trading Volume vs. Market Cap",
    x_axis_type='log',
    y_axis_type='log',
    x_axis_label='Trading Volume (log)',
    y_axis_label='Market Capitalization (log)'
    
)

p.scatter(
    x='market_cap',
    y='volume_24h',
    source=ColumnDataSource(cap_vol),
    size=5,
    alpha=0.4,
    color=linear_cmap(field_name='volatility_7d', palette=Spectral6, low=-1, high=10)
)

p.add_tools(HoverTool(tooltips=[
    ('name','@name'),
    ('market cap', '@market_cap{$ 0.00 a}'),
    ('trading_volume', '@volume_24h'),
    ('volatility (7 days)', '@volatility_7d{0.0%}')
],
))

show(p)